In [ ]:
import pandas as pd

In [ ]:
# Clone repo
! git clone https://github.com/dennybritz/cnn-text-classification-tf.git

Cloning into 'cnn-text-classification-tf'...
remote: Enumerating objects: 234, done.
remote: Total 234 (delta 0), reused 0 (delta 0), pack-reused 234
Receiving objects: 100% (234/234), 1.11 MiB | 25.28 MiB/s, done.
Resolving deltas: 100% (121/121), done.


In [ ]:
# Downgrade Python (TF 2.x not supported for python >= 3.8)
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.7 1

In [ ]:
!sudo update-alternatives --config python

update-alternatives: error: no alternatives for python


In [ ]:
# Install modules
! pip install tensorflow==1.13.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 92.6 MB 78 kB/s 
     |████████████████████████████████| 50 kB 8.9 MB/s 
     |████████████████████████████████| 367 kB 67.1 MB/s 
     |████████████████████████████████| 3.2 MB 70.7 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.9.0
    Uninstalling tensorflow-estimator-2.9.0:
      Successfully uninstalled tensorflow-estimator-2.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.8.0+zzzcolab20220506162203
    Uninstalling tensorflow-2.8.0+zzzcolab20220506162203:
      Successfully uninstalled tensorflow-2.8.0+zzzcolab20220506162203
ERROR: pip's dependency resolver does not currently tak

In [ ]:
! python cnn-text-classification-tf/train.py

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

In [ ]:
! zip -r runs.zip runs/

  adding: runs/ (stored 0%)
  adding: runs/1669754187/ (stored 0%)
  adding: runs/1669754187/summaries/ (stored 0%)
  adding: runs/1669754187/summaries/dev/ (stored 0%)
  adding: runs/1669754187/summaries/dev/events.out.tfevents.1669754187.276e31bbb49e (deflated 88%)
  adding: runs/1669754187/summaries/train/ (stored 0%)
  adding: runs/1669754187/summaries/train/events.out.tfevents.1669754187.276e31bbb49e (deflated 88%)
  adding: runs/1669754187/vocab (deflated 54%)
  adding: runs/1669754187/checkpoints/ (stored 0%)
  adding: runs/1669750826/ (stored 0%)
  adding: runs/1669750826/summaries/ (stored 0%)
  adding: runs/1669750826/summaries/dev/ (stored 0%)
  adding: runs/1669750826/summaries/dev/events.out.tfevents.1669750826.276e31bbb49e (deflated 88%)
  adding: runs/1669750826/summaries/train/ (stored 0%)
  adding: runs/1669750826/summaries/train/events.out.tfevents.1669750826.276e31bbb49e (deflated 86%)
  adding: runs/1669750826/vocab (deflated 54%)
  adding: runs/1669750826/checkpoin

In [ ]:
# distilBERT-based classification

# Upgrade Python
! sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.7 1
! pip install tensorflow==2.8.0
! pip install transformers==4.6.0
! pip install keras==2.8.0

update-alternatives: error: alternative path /usr/bin/python3.7 doesn't exist
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 497.6 MB 1.7 kB/s 
     |████████████████████████████████| 1.4 MB 66.3 MB/s 
     |████████████████████████████████| 462 kB 76.8 MB/s 
     |████████████████████████████████| 5.8 MB 50.6 MB/s 
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.9.1
    Uninstalling tensorboard-2.9.1:
      Successfully uninstalled tensorboard-2.9.1
  Attempting uninstall: keras
    Found existing installation: keras 2.9.0
    Uninstalling keras-2.9.0:
      Successfully uninstalled keras-2.9.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.9.2
    Uninstalling tensorflow-2.9.2:
      Successfully uninstalled tensorflow-2.9.2
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
   

In [ ]:
# Load post data
posts = pd.read_json("data_labeled_lf_2class.json")

# Exclude posts with 0/empty comments
posts = posts[posts["Comments Text"].apply(lambda x: len(x) > 0)]

In [ ]:
import tensorflow as tf
from transformers import DistilBertTokenizer, AutoTokenizer, DataCollatorWithPadding, AutoModelForSequenceClassification, TrainingArguments, Trainer, create_optimizer
from sklearn.model_selection import train_test_split

# Tokenize text
text_post = list(posts["Text"])
text_comments = [" ".join(text) for text in posts["Comments Text"]]

tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased", truncation=True, padding=True)

def fn_tokenize(texts):
    return tokenizer(texts, truncation=True)

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
unique_labels = set(label for label in posts["label_predicted"])

X_train, X_test, y_train, y_test = train_test_split(text_post, list(posts["label_predicted"]), test_size=0.2)

In [ ]:
train_encodings = tokenizer(X_train, is_split_into_words=False, padding=True, truncation=True)
val_encodings = tokenizer(X_test, is_split_into_words=False, padding=True, truncation=True)

train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    y_train
))
val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    y_test
))

In [ ]:
from transformers import TFDistilBertForSequenceClassification, TFTrainer, TFTrainingArguments
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

BATCH_SIZE = 16
model = TFDistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")#chose the optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)#define the loss function 
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)#build the model
model.compile(optimizer=optimizer,
              loss=loss,
              metrics=['accuracy', precision_m, recall_m, f1_m])# train the model 
model.fit(train_dataset.shuffle(len(X_train)).batch(BATCH_SIZE),
          validation_data=val_dataset.batch(BATCH_SIZE),
          epochs=10,
          batch_size=BATCH_SIZE)

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_projector', 'vocab_layer_norm', 'activation_13', 'vocab_transform']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['dropout_39', 'pre_classifier', 'classifier']
You should probably TRAIN this model on a down-stream task to be able to use i

Epoch 1/10


206/206 [==============================] - ETA: 0s - loss: 0.4191 - accuracy: 0.8239 - precision_m: 0.1535 - recall_m: 0.5583 - f1_m: 0.2314

206/206 [==============================] - 214s 999ms/step - loss: 0.4191 - accuracy: 0.8239 - precision_m: 0.1535 - recall_m: 0.5583 - f1_m: 0.2314 - val_loss: 0.3401 - val_accuracy: 0.8819 - val_precision_m: 0.1501 - val_recall_m: 0.4973 - val_f1_m: 0.2174
Epoch 2/10
206/206 [==============================] - 204s 990ms/step - loss: 0.3049 - accuracy: 0.8794 - precision_m: 0.1678 - recall_m: 0.6427 - f1_m: 0.2561 - val_loss: 0.3001 - val_accuracy: 0.8831 - val_precision_m: 0.1796 - val_recall_m: 0.6650 - val_f1_m: 0.2707
Epoch 3/10
206/206 [==============================] - 204s 992ms/step - loss: 0.2361 - accuracy: 0.9092 - precision_m: 0.1722 - recall_m: 0.6703 - f1_m: 0.2636 - val_loss: 0.3221 - val_accuracy: 0.8855 - val_precision_m: 0.1868 - val_recall_m: 0.8009 - val_f1_m: 0.2910
Epoch 4/10
206/206 [==============================] - 205s 993ms/step - loss: 0.1613 - accuracy: 0.9433 - precision_m: 0.1965 - recall_m: 0.7978 - f1_m: 0.3050 - val_loss: 0.3265 - val_accuracy: 0.8733